In [ ]:
import requests
from requests import Response
from pydantic import BaseModel,RootModel,Field,field_validator,field_serializer
from datetime import datetime
from pprint import pprint

url='https://data.ntpc.gov.tw/api/datasets/c51d5111-c300-44c9-b4f1-4b28b9929ca2/json?size=1000'
try:
    respons:Response=requests.request('GET',url)
    respons.raise_for_status()
except Exception:
    print("下載錯誤")
else:
    content=respons.text

#使用自訂的Fields的名稱
#轉換型別
#自訂驗証
#自訂輸出

class Station(BaseModel):
    名稱:str=Field(alias='organizer')
    註冊編號:int=Field(alias='no')
    地址:str=Field(alias='address')
    統編:int=Field(alias='tax_id_number')
    緯度:float=Field(alias='wgs84ax')
    經度:float=Field(alias='wgs84ay')
    日期:datetime=Field(alias='available_return_bikes')

    @field_validator('站名')
    @classmethod
    def slice_str(cls,name:str):
        return name[11:]
    
    @field_validator('狀態',mode="before")
    @classmethod
    def slice_str(cls,status:str):
        if status=="1":
            return "營業中"
        else:
            return "維修中"
        
    @field_serializer("日期")
    def date_serializer(self,date:datetime)->str:
        return date.strftime(f"中華民國{date.year-1911}年%m月%d日 %H時%M分%S秒")
class Youbikes(RootModel):
    root:list[Station]

youbikes:Youbikes=Youbikes.model_validate_json(content)
youbikes_list=youbikes.model_dump()
pprint(youbikes_list)